In [ ]:
# Cosmic rays (loop removed)
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from random import randint, uniform

# Specified Characteristics
class SyntheticRamanSpectrum:
    def __init__(self, pixels, poly_order_range, num_peaks_range, snr_range, cosmic_ray_prob, num_spectra):
        self.pixels = pixels
        self.poly_order_range = poly_order_range
        self.num_peaks_range = num_peaks_range
        self.snr_range = snr_range
        self.cosmic_ray_prob = cosmic_ray_prob
        self.num_spectra = num_spectra
        self.wavelengths = np.linspace(0, 1, self.pixels)
        self.spectrum = np.zeros(self.pixels)  # Initialize with zeros
        self.generate_spectrum()  # Update spectrum

# Generate Baseline
    def generate_spectrum(self):
        # Generate baseline using a polynomial
        poly_order = np.random.randint(self.poly_order_range[0], self.poly_order_range[1] + 1)
        baseline_coefficients = np.random.rand(poly_order + 1)
        baseline = np.polyval(baseline_coefficients, self.wavelengths)
        
        return baseline
        
# Generate Lorentzian peaks
        peaks = np.zeros_like(self.wavelengths)
        num_peaks = np.random.randint(self.num_peaks_range[0], self.num_peaks_range[1] + 1)
        
        for _ in range(num_peaks):
            peak_amplitude = np.random.uniform(0, 1)
            peak_position = np.random.uniform(0.2, 0.8)
            peak_width = 1 / (self.pixels * 0.1)
            peaks += peak_amplitude * (peak_width ** 2) / ((self.wavelengths - peak_position) ** 2 + peak_width ** 2)
        
        # Add peaks to signal
        peaky_spectrum = baseline + peaks
        
        return peaky_spectrum
        
# Generate Gaussian noise
        noise_amplitude_range = (0.1, 0.5)
        # Calculate signal power
        signal_power = np.mean(np.abs(peaky_spectrum) ** 2)
    
        # Calculate noise power based on SNR
        noise_power = signal_power / self.snr_range
    
        # Calculate standard deviation of noise
        noise_std = np.sqrt(noise_power)
        
        # Generate random noise amplitudes within specified range
        min_amp, max_amp = noise_amplitude_range
        noise_amplitudes = np.random.uniform(min_amp, max_amp)
    
        # Generate Gaussian noise samples
        noise = np.random.normal(0, noise_std, len(self.wavelengths))
    
        # Scale the noise by noise amplitudes
        noise *= noise_amplitudes
    
        # Add noise to signal
        noisy_spectrum = peaky_spectrum + noise
        
        return noisy_spectrum
         
    def chebyshev_polynomial(self, x, *coefficients):
        n = len(coefficients)
        chebyshev_terms = [coefficients[i] * np.polynomial.chebyshev.chebval(x, (i+1)) for i in range(n)]
        baseline_variation = np.sum(chebyshev_terms, axis=0)

        # Add Chebyshev variation to signal
        chebyshev_spectrum = self.spectrum + baseline_variation

        return chebyshev_spectrum

    def add_cosmic_rays(self, num_cosmic_rays):
        cosmic_rays = self.chebyshev_polynomial(self.wavelengths)
    
        # Calculate L2-norm of complete signal
        l2_norm = np.linalg.norm(self.spectrum)
    
        for _ in range(num_cosmic_rays):
            # Generate random amplitude between 0.8 & l2_norm
            cosmic_ray_amplitude = np.random.uniform(0.8, l2_norm)
        
            # Randomly select position for cosmic ray
            position = np.random.randint(0, len(self.spectrum))
        
            # Add cosmic ray with width zero
            cosmic_rays_position = position + cosmic_ray_amplitude
            
            # Add Chebyshev variation to signal
            cosmic_rays += cosmic_rays_position * self.chebyshev_polynomial(self.wavelengths)
        
        return cosmic_rays

# Define number of cosmic rays
num_cosmic_rays = 5 
    
# Create an instance of SyntheticRamanSpectrum class
synthetic_spectrum = SyntheticRamanSpectrum(
    pixels=1024,
    poly_order_range=(2, 4),
    num_peaks_range=(2, 5),
    snr_range=10,
    cosmic_ray_prob=0.1,
    num_spectra=1
)

# Generate the noisy_spectrum
noisy_spectrum = synthetic_spectrum.generate_spectrum()
# Add cosmic rays to spectrum by passing noisy_spectrum as an argument
cosmic_rays_spectrum = synthetic_spectrum.add_cosmic_rays(num_cosmic_rays)

# Plot spectrum with cosmic rays
plt.figure(figsize=(10, 6))
plt.plot(synthetic_spectrum.wavelengths, cosmic_rays_spectrum, label="Spectrum with Cosmic Rays", color="orange")
plt.xlabel("Wavelength")
plt.ylabel("Intensity")
plt.title("Cosmic Rays")
plt.legend()
plt.grid(True)
plt.show()